# SELECT within SELECT

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Bigger than Russia

**List each country name where the population is larger than that of 'Russia'.**

```
world(name, continent, area, population, gdp)
```

In [2]:
world <- dbReadTable(con, 'world')

In [3]:
bm <- world %>% 
    filter(name=='Russia') %>% 
    select(population) %>% pull
world %>%
    filter(population > bm) %>% 
    select(name)

name
<chr>
Bangladesh
Brazil
China
India
Indonesia
Nigeria
Pakistan
United States


## 2. Richer than UK

**Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.**

> _Per Capita GDP_   
> The per capita GDP is the gdp/population

In [4]:
bm <- world %>% 
    filter(name=='United Kingdom') %>% 
    mutate(pc_gdp=gdp/population) %>% 
    select(pc_gdp) %>% 
    pull
world %>% 
    mutate(pc_gdp=gdp/population) %>% 
    filter(continent=='Europe' & pc_gdp > bm) %>%
    select(name)

name
<chr>
Andorra
Austria
Belgium
Denmark
Finland
France
Germany
Iceland
Ireland


## 3. Neighbours of Argentina and Australia


List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

In [5]:
bm <- world %>%
    filter(name %in% c('Argentina', 'Australia')) %>% 
    select(continent) %>% 
    distinct %>% 
    pull
world %>% 
    filter(continent %in% bm) %>%
    arrange(name) %>% 
    select(name, continent)

name,continent
<chr>,<chr>
Argentina,South America
Australia,Oceania
Bolivia,South America
Brazil,South America
Chile,South America
Colombia,South America
Ecuador,South America
Fiji,Oceania
Guyana,South America


## 4. Between Canada and Poland

Which country has a population that is more than Canada but less than Poland? Show the name and the population.

In [6]:
bm <- world %>% 
    filter(name %in% c('Canada', 'Poland')) %>% 
    select(population) %>% 
    pull %>%
    set_names(c('Canada', 'Poland'))
world %>% 
    filter(population > bm['Canada'] &
           population < bm['Poland']) %>% 
    select(name, population)

name,population
<chr>,<dbl>
Iraq,36004552
Sudan,37289406


## 5. Percentages of Germany

Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

**Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.**

The format should be Name, Percentage for example:

name	| percentage
--------|-----------
Albania	| 3%
Andorra	| 0%
Austria	| 11%
...	| ...

> _Decimal places_   
> You can use the function ROUND to remove the decimal places.

> _Percent symbol %_
> You can use the function CONCAT to add the percentage symbol.

[To get a well rounded view of the important features of SQL you should move on to the next tutorial concerning aggregates.](https://sqlzoo.net/wiki/SUM_and_COUNT)

To gain an absurdly detailed view of one insignificant feature of the language, read on.

We can use the word `ALL` to allow >= or > or < or <=to act over a list. For example, you can find the largest country in the world, by population with this query:

```sql
SELECT name
  FROM world
 WHERE population >= ALL(SELECT population
                           FROM world
                          WHERE population>0)
```

You need the condition **population>0** in the sub-query as some countries have **null** for population.

In [7]:
bm <- world %>% 
    filter(name=='Germany') %>% 
    select(population) %>%
    pull
world %>% 
    filter(continent=='Europe') %>% 
    mutate(percentage=sprintf('%0.0f%%', 100*round(population/bm, 2))) %>% 
    select(name, percentage)

name,percentage
<chr>,<chr>
Albania,3%
Andorra,0%
Austria,11%
Belarus,12%
Belgium,14%
Bosnia and Herzegovina,5%
Bulgaria,9%
Croatia,5%
Czech Republic,13%


## 6. Bigger than every country in Europe

Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

We can refer to values in the outer SELECT within the inner SELECT. We can name the tables so that we can tell the difference between the inner and outer versions.

In [8]:
bm <- world %>% 
    filter(continent=='Europe' & ! is.na(gdp)) %>% 
    select(gdp) %>% 
    pull
world %>% 
    filter(gdp %>% (function(x) {
        vapply(x, function(i) all(i>bm), logical(1L))
    })) %>% 
    select(name)

name
<chr>
China
Japan
United States


## 7. Largest in each continent

**Find the largest country (by area) in each continent, show the continent, the name and the area:**

```sql
SELECT continent, name, population FROM world x
  WHERE population >= ALL
    (SELECT population FROM world y
        WHERE y.continent=x.continent
          AND population>0)
```

> __The above example is known as a correlated or synchronized sub-query.__   
> 
> Using correlated subqueries
> A correlated subquery works like a nested loop: the subquery only has access to rows related to a single > record at a time in the outer query. The technique relies on table aliases to identify two different uses of the same table, one in the outer query and the other in the subquery.
> 
> One way to interpret the line in the **WHERE** clause that references the two table is _“… where the correlated values are the same”._
> 
> In the example provided, you would say _“select the country details from world where the population is greater than or equal to the population of all countries where the continent is the same”._

In [9]:
world %>% 
    group_by(continent) %>% 
    top_n(1, area) %>% 
    select(continent, name, area)

continent,name,area
<chr>,<chr>,<dbl>
Africa,Algeria,2381741
Oceania,Australia,7692024
South America,Brazil,8515767
North America,Canada,9984670
Asia,China,9596961
Caribbean,Cuba,109884
Europe,Kazakhstan,2724900
Eurasia,Russia,17125242


## 8. First country of each continent (alphabetically)

**List each continent and the name of the country that comes first alphabetically.**

In [10]:
world %>% 
    group_by(continent) %>% 
    slice(1) %>% 
    select(continent, name)

continent,name
<chr>,<chr>
Africa,Algeria
Asia,Afghanistan
Caribbean,Antigua and Barbuda
Eurasia,Armenia
Europe,Albania
North America,Belize
Oceania,Australia
South America,Argentina


## 9. Difficult Questions That Utilize Techniques Not Covered In Prior Sections

**Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.**

In [11]:
bm <- world %>% 
    group_by(continent) %>% 
    mutate(flg=population<=25000000) %>% 
    filter(sum(flg)==n()) %>% 
    pull(continent) %>% 
    unique
world %>% 
    filter(continent %in% bm) %>% 
    select(name, continent, population)

name,continent,population
<chr>,<chr>,<dbl>
Antigua and Barbuda,Caribbean,86295
Australia,Oceania,23545500
Bahamas,Caribbean,351461
Barbados,Caribbean,285000
Cuba,Caribbean,11167325
Dominica,Caribbean,71293
Dominican Republic,Caribbean,9445281
Fiji,Oceania,858038
Grenada,Caribbean,103328


## 10.

**Some countries have populations more than three times that of any of their neighbours (in the same continent). Give the countries and continents.**

In [12]:
# self full join other countries with population*3
# filter out population > other population*3
# count the countries, search for n=(country number on the continent)-1
world %>% 
    select(name, continent, population) %>% 
    full_join(world %>% 
              mutate(popl3=population*3) %>% 
              select(continent, name, popl3), 
              by='continent', 
              suffix=c('', '.y')) %>% 
    filter(name != name.y, population > popl3) %>%
    group_by(continent, name) %>% 
    summarise(n=n()) %>%
    inner_join(world %>% 
               group_by(continent) %>%
               summarise(n=n()-1), 
               by='continent', 
               suffix=c('', '.y')) %>%
    filter(n==n.y) %>% 
    select(name, continent)

name,continent
<chr>,<chr>
Russia,Eurasia
Australia,Oceania
Brazil,South America


In [13]:
dbDisconnect(con)